# Query Data with LLM using Google Gemini 

In this section, we will review different ways to query data with LLMs using the [Gemini API](https://ai.google.dev/gemini-api/docs). We will start with basic prompt methods to generate SQL queries and then move on to more advanced prompt techniques. We than fully functionalize the process and build the following three components:

- **Create Prompt**: This function takes a user question and returns a prompt
- **API handler**: This function sends prompts to the API, returns the response, and parse it to SQL query
- **DB handler**: This function takes the query and returns a dataframe 


<figure>
 <img src="images/architecture.png" width="100%" align="center"/></a>
<figcaption> The agent architecture </figcaption>
</figure>

<br>
<br />

To demonstrate the process, we will use the Chicago Crime dataset we pulled earlier from the API and load it into a Pandas dataframe. We will also use the `duckdb` library to query the data using SQL queries.

<figure>
 <img src="images/chicago_crime.png" width="100%" align="center"/></a>
<figcaption> The Chicago Crime dataset </figcaption>
</figure>

<br>
<br />

In [161]:
import pandas as pd
import duckdb 
import os
from google import genai
from google.genai import types

In [162]:
chicago_crime = pd.read_csv("./data/chicago_crime_2023_2025.csv")
chicago_crime["updated_on"] = pd.to_datetime(chicago_crime["updated_on"])
chicago_crime["x_coordinate"] = chicago_crime["x_coordinate"].astype(float)
chicago_crime["y_coordinate"] = chicago_crime["y_coordinate"].astype(float)
chicago_crime["latitude"] = chicago_crime["latitude"].astype(float)
chicago_crime["longitude"] = chicago_crime["longitude"].astype(float)
chicago_crime["year"]= chicago_crime["year"].astype(int)

In [163]:
chicago_crime.head()

,id,case_number,datetime,block,iucr,primary_type,description,location_description,arrest,domestic,...,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude
0,13140855,JG341458,2023-01-01 00:00:00,082XX S JEFFERY BLVD,1754,OFFENSE INVOLVING CHILDREN,AGGRAVATED SEXUAL ASSAULT OF CHILD BY FAMILY M...,APARTMENT,False,True,...,4,8.0,46.0,02,1190953.0,1850848.0,2023,2023-09-24 15:41:26,41.745739,-87.575883
1,13180096,JG387858,2023-01-01 00:00:00,075XX S WOLCOTT AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,6,17.0,71.0,11,1164996.0,1854651.0,2023,2023-08-20 15:40:56,41.756762,-87.670887
2,13168471,JG374193,2023-01-01 00:00:00,013XX W HARRISON ST,0460,BATTERY,SIMPLE,SCHOOL - PUBLIC GROUNDS,False,False,...,12,34.0,28.0,08B,1167465.0,1897475.0,2023,2023-08-19 15:40:26,41.874223,-87.660610
3,13078152,JG267031,2023-01-01 00:00:00,101XX S BEVERLY AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,22,21.0,73.0,11,1168800.0,1837525.0,2023,2023-08-19 15:40:26,41.709685,-87.657439
4,13120699,JG314178,2023-01-01 00:00:00,063XX N FAIRFIELD AVE,1544,SEX OFFENSE,SEXUAL EXPLOITATION OF A CHILD,OTHER (SPECIFY),False,False,...,24,50.0,2.0,17,1156847.0,1941985.0,2023,2023-08-19 15:40:26,41.996584,-87.698384


In [164]:
print(chicago_crime.dtypes)

id                               int64
case_number                     object
datetime                        object
block                           object
iucr                            object
primary_type                    object
description                     object
location_description            object
arrest                            bool
domestic                          bool
beat                             int64
district                         int64
ward                           float64
community_area                 float64
fbi_code                        object
x_coordinate                   float64
y_coordinate                   float64
year                             int64
updated_on              datetime64[ns]
latitude                       float64
longitude                      float64
dtype: object


## The Gemini API

Throughout this workshop, we will use API as the backend LLM for generating the SQL queries. This notebook will show how to use the Gemini API for generating SQL queries. Let's start by reviewing the functionality of the API.

### Setting Up API Key

To use the Gemini API, you will have to register to the API and setup your own key. To register to the API go the [Gemini Developer API main page](https://ai.google.dev/gemini-api/docs), and click on the "Get a Gemini API Key" button on the top right corner (marked in purple):

<figure>
 <img src="images/gemini_api.png" width="100%" align="center"/></a>
<figcaption> The Gemini Developer API page </figcaption>
</figure>

<br>
<br />

If you already have a Google account it will direct you to the API Key page, where you can create keys (marked in yellow):
<figure>
 <img src="images/gemini_api_key.png" width="100%" align="center"/></a>
<figcaption> Creating a new API key </figcaption>
</figure>

<br>
<br />
You can see in the button of this page the list of existing API keys. Note that we are using a free plan, therefore, we will be limited in the type of models that we can use.


You should store the API key as an environment variable (for macOS and Linux) by adding the following line to your code to your `zshrc` or `bashrc` file:

```bash
export GEMINI_API_KEY="your_api_key"
```

<br>
<br />

## Getting Started with the Gemini API

Let's start by creating a client object by registering the API key. We will load the environment variable using the `os`'s `environ.get()` method: 

In [165]:
api_key = os.environ.get("GEMINI_API_KEY")
client = genai.Client(api_key = api_key)

We will use the Gemini text generating model using the `models.generate_content` [method](https://ai.google.dev/api/generate-content#method:-models.generatecontent) to send prompts to the API. Here are the key arguments of the function:
- `model`: The name of the model to use, in this case, [gemini-2.0-flash](https://ai.google.dev/gemini-api/docs/models#gemini-2.0-flash).
- `content`: The prompt for the model to generate content.
- `max_output_tokens`: The maximum number of tokens to include in a response candidate.
- `temperature`: Controls the randomness of the output. Higher values result in more varied and creative outputs, while lower values lead to more predictable and conservative responses.

Throughout this workshop, to reproduce the same results, we will set the `temperature` argument to 0.

We will start with a simplistic prompt, where we are going to provides the model with context and ask it to generate a query. The Gemini API  specification requires us to provide the `contents` argument with your prompt. We will start by simply define a the following prompt:

In [166]:
prompt1 = """
        I am working with a dataset that contains information about Chicago crime incidents. 
        I want to create a SQL query to pull the total number of crimes that ended in arrest by a year
        """

In [167]:
response1 = client.models.generate_content(
    model="gemini-2.0-flash",
    contents= prompt1,
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0
    )
)

Let's print the response:

In [168]:
print(response1.candidates[0].content.parts[0].text)

```sql
SELECT
    STRFTIME('%Y', Date) AS CrimeYear,  -- Extract the year from the 'Date' column
    COUNT(*) AS TotalArrests  -- Count the number of crimes in each year that resulted in an arrest
FROM
    ChicagoCrimeData  -- Replace 'ChicagoCrimeData' with the actual name of your table
WHERE
    Arrest = TRUE  -- Filter for incidents where an arrest was made
GROUP BY
    CrimeYear  -- Group the results by year
ORDER BY
    CrimeYear;  -- Order the results by year for readability
```

**Explanation:**

1.  **`SELECT STRFTIME('%Y', Date) AS CrimeYear`**:
    *   `STRFTIME('%Y', Date)`: This extracts the year from the `Date` column.  The `STRFTIME` function is a SQLite function (and often available in other SQL dialects) that allows you to format date and time values.  `'%Y'` specifies that you want to extract the year in a four-digit format (e.g., 2023).
    *   `AS CrimeYear`: This assigns the alias "CrimeYear" to the extracted year, making it easier to refer to in the rest of the que

As you can notice, the model response is generic and due to a lack of context. The model makes some assumptions about the table's column names and their attributes. We can improve this by providing a better context in the next example:

In [169]:
prompt2 = """I am working with a dataset that contains information about Chicago crime incidents. 
        The table name is 'chicago_crime'. I want to create a SQL query to pull the total number of crimes that ended in arrest by a year. 
        The columns are: id, case_number, date, block, primary_type, description, location_description, arrest, domestic, beat, district, ward, community_area, fbi_code, x_coordinate, y_coordinate, year, updated_on. Please provide the SQL query.
        """

In [170]:
response2 = client.models.generate_content(
    model="gemini-2.0-flash",
    contents= prompt2,
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0
    )
)

In [171]:
print(response2.candidates[0].content.parts[0].text)

```sql
SELECT
    year,
    COUNT(*) AS total_arrests
FROM
    chicago_crime
WHERE
    arrest = TRUE  -- Assuming 'arrest' is a boolean column (TRUE/FALSE)
    -- If 'arrest' is a string column ('TRUE'/'FALSE' or 'Yes'/'No'), adjust the WHERE clause accordingly.
    -- For example: WHERE arrest = 'TRUE' or WHERE arrest = 'Yes'
GROUP BY
    year
ORDER BY
    year;
```

**Explanation:**

1. **`SELECT year, COUNT(*) AS total_arrests`**:  This selects the `year` and the count of all rows for each year, aliasing the count as `total_arrests`.
2. **`FROM chicago_crime`**:  This specifies the table to retrieve data from.
3. **`WHERE arrest = TRUE`**: This filters the data to include only rows where the `arrest` column indicates an arrest was made.  **Important:**  This assumes your `arrest` column is a boolean (TRUE/FALSE).  If it's a string (e.g., 'TRUE', 'FALSE', 'Yes', 'No'), you'll need to adjust this condition accordingly (see the comment in the code).
4. **`GROUP BY year`**: This groups 

Let's copy the SQL code and test it:

In [172]:
sql = """
SELECT
    year,
    COUNT(*) AS total_arrests
FROM
    chicago_crime
WHERE
    arrest = TRUE  -- Assuming 'arrest' is a boolean column (TRUE/FALSE)
    -- If 'arrest' is a string column ('TRUE'/'FALSE' or 'Yes'/'No'), adjust the WHERE clause accordingly.
    -- For example: WHERE arrest = 'TRUE' or WHERE arrest = 'Yes'
GROUP BY
    year
ORDER BY
    year;
    """

In [173]:
duckdb.sql(sql).show()

┌───────┬───────────────┐
│ year  │ total_arrests │
│ int64 │     int64     │
├───────┼───────────────┤
│  2023 │         31990 │
│  2024 │         35376 │
│  2025 │         12555 │
└───────┴───────────────┘



## Setting Templates 

In this section we will generalize the prompt by setting a prompt template. Starting by setting a dedicated template for the Chicago Crime dataset:

In [174]:
prompt_template = """I am working with a dataset that contains information about Chicago crime incidents. 
                    The table name is 'chicago_crime'. I want to create a SQL query that answers the following question: "{}". 
                    The columns are: id, case_number, date, block, primary_type, description, location_description, arrest, domestic, beat, district, ward, community_area, fbi_code, x_coordinate, y_coordinate, year, updated_on. Please provide the SQL query.
                """

In [175]:
user_question = "How many cases ended up with arrest during 2024?"
prompt3 = prompt_template.format(user_question)

In [176]:
response3 = client.models.generate_content(
    model="gemini-2.0-flash",
    contents= prompt3,
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0
    )
)

In [177]:
print(response3.candidates[0].content.parts[0].text)

```sql
SELECT COUNT(*) AS arrest_count
FROM chicago_crime
WHERE year = 2024
  AND arrest = TRUE;
```

**Explanation:**

*   **`SELECT COUNT(*) AS arrest_count`**: This counts all rows that meet the specified criteria and names the resulting count `arrest_count`.
*   **`FROM chicago_crime`**: This specifies that we are querying the `chicago_crime` table.
*   **`WHERE year = 2024`**: This filters the data to include only incidents that occurred in the year 2024.
*   **`AND arrest = TRUE`**: This further filters the data to include only incidents where an arrest was made.  The `arrest` column is assumed to be a boolean (TRUE/FALSE) indicating whether an arrest occurred.  If the `arrest` column is stored as a different data type (e.g., 'Y'/'N', 1/0), you'll need to adjust this condition accordingly (e.g., `AND arrest = 'Y'` or `AND arrest = 1`).



Let's evaluate the query with the `duckdb` `sql` method:

In [178]:
sql = """
SELECT COUNT(*) AS arrest_count
FROM chicago_crime
WHERE year = 2024
  AND arrest = TRUE;
"""

In [179]:
duckdb.sql(sql).show()

┌──────────────┐
│ arrest_count │
│    int64     │
├──────────────┤
│        35376 │
└──────────────┘



## Generalize the Template

Let's now generalize the template that it can be applied on any table. We will split the template into two parts: one for the user and another for the system. The system part will be a prompt that describes the table's structure, while the user part will ask for a query. 

**Note:** We will follow the same naming convention as in the OpenAI's example (i.e., `system` and `user` roles), and concatenate the two parts to form the final prompt.

Here is the system template:

In [180]:
system_template = """
Given the following SQL table, your job is to write queries given a user’s request. \n
CREATE TABLE {} ({}) \n
"""

Where, we will define the table schema as SQL statement. Likewise, we will define the user template as:

In [181]:
user_template = "Write a SQL query that returns - {}"

Next, we want to auto generate the table schema the by applying SQL `DESCRIBE` command leveraging `duckdb` SQL execution on the table:

In [182]:
table_name = "chicago_crime" 
tbl_description = duckdb.sql("DESCRIBE SELECT * FROM " + table_name +  ";")
tbl_description

┌──────────────────────┬──────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name      │ column_type  │  null   │   key   │ default │  extra  │
│       varchar        │   varchar    │ varchar │ varchar │ varchar │ varchar │
├──────────────────────┼──────────────┼─────────┼─────────┼─────────┼─────────┤
│ id                   │ BIGINT       │ YES     │ NULL    │ NULL    │ NULL    │
│ case_number          │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ datetime             │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ block                │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ iucr                 │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ primary_type         │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ description          │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ location_description │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ arrest               │ BOOLEAN      │ 

We will parse from the table description the column names and their class:

In [183]:
col_attr = tbl_description.df()[["column_name", "column_type"]]
col_attr["column_joint"] = col_attr["column_name"] + " " +  col_attr["column_type"]
col_names = str(list(col_attr["column_joint"].values)).replace('[', '').replace(']', '').replace('\'', '')
col_names

'id BIGINT, case_number VARCHAR, datetime VARCHAR, block VARCHAR, iucr VARCHAR, primary_type VARCHAR, description VARCHAR, location_description VARCHAR, arrest BOOLEAN, domestic BOOLEAN, beat BIGINT, district BIGINT, ward DOUBLE, community_area DOUBLE, fbi_code VARCHAR, x_coordinate DOUBLE, y_coordinate DOUBLE, year BIGINT, updated_on TIMESTAMP_NS, latitude DOUBLE, longitude DOUBLE'

We can now use it to set the table schema on the system template:

In [184]:
system = system_template.format(table_name, col_names)
print(system)


Given the following SQL table, your job is to write queries given a user’s request. 

CREATE TABLE chicago_crime (id BIGINT, case_number VARCHAR, datetime VARCHAR, block VARCHAR, iucr VARCHAR, primary_type VARCHAR, description VARCHAR, location_description VARCHAR, arrest BOOLEAN, domestic BOOLEAN, beat BIGINT, district BIGINT, ward DOUBLE, community_area DOUBLE, fbi_code VARCHAR, x_coordinate DOUBLE, y_coordinate DOUBLE, year BIGINT, updated_on TIMESTAMP_NS, latitude DOUBLE, longitude DOUBLE) 




Likewise, we will set the user template by adding the question:

In [185]:
question = "How many cases ended up with arrest during 2024?"
user = user_template.format(question)
print(user)

Write a SQL query that returns - How many cases ended up with arrest during 2024?


In [186]:
prompt4 = [system, user]

In [187]:
response4 = client.models.generate_content(
    model="gemini-2.0-flash",
    contents= prompt4,
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0
    )
)

In [188]:
print(response4.candidates[0].content.parts[0].text)

```sql
SELECT
  COUNT(CASE WHEN arrest = TRUE THEN 1 ELSE NULL END)
FROM chicago_crime
WHERE
  year = 2024;
```


Note that the return SQL code comes in markdown code chunk format (e.g., `'''sql`). While with the OpenAI models it was sufficient to modify the prompt, it does not seems to be effective with the Gemini model. We will have to use a custom function to extract the code from the response:

In [189]:
import re

def is_markdown_code_chunk(text):
    """
    Checks if the given text is in Markdown code chunk format.

    Args:
        text (str): The text to check.

    Returns:
        bool: True if the text is in Markdown code chunk format, False otherwise.
    """
    pattern = r"```[^`]*```"
    return bool(re.search(pattern, text, re.DOTALL))

def extract_code_from_markdown(markdown_text):
    """
    Extracts code from a Markdown code chunk.

    Args:
        markdown_text (str): The Markdown text containing the code chunk.

    Returns:
        str: The extracted code.
    """
    pattern = r"```(.*?)\n(?P<code>.*?)\n```"
    match = re.search(pattern, markdown_text, re.DOTALL)
    if match:
        return match.group("code")
    else:
        return None




The first function - `is_markdowdown_code_chunk` is a function that checks if the input text contains code in Markdown format. It uses regular expressions to match the pattern of a Markdown code chunk, which consists of three backticks. The second function - `extract_code_from_markdown` extract the code from the Markdown format.

Let's see an example of how to use these functions:

In [190]:
sql_chunk  = response4.candidates[0].content.parts[0].text

if is_markdown_code_chunk(text= sql_chunk):
    sql = extract_code_from_markdown(markdown_text= sql_chunk)
else:
    sql = sql_chunk

print(sql_chunk)
print(sql)

```sql
SELECT
  COUNT(CASE WHEN arrest = TRUE THEN 1 ELSE NULL END)
FROM chicago_crime
WHERE
  year = 2024;
```
SELECT
  COUNT(CASE WHEN arrest = TRUE THEN 1 ELSE NULL END)
FROM chicago_crime
WHERE
  year = 2024;


Let's evaluate the sql code:

In [191]:
duckdb.sql(sql).show()

┌────────────────────────────────────────────────────────────────────────────┐
│ count(CASE  WHEN ((arrest = CAST('t' AS BOOLEAN))) THEN (1) ELSE NULL END) │
│                                   int64                                    │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                      35376 │
└────────────────────────────────────────────────────────────────────────────┘



## Functionalize the Process

The last step is to create a function that takes a user question and returns the corresponding data. The `create_message` function set the system and user message (or prompt) template:

<figure>
 <img src="images/architecture 1.png" width="100%" align="center"/></a>
<figcaption> The agent architecture</figcaption>
</figure>

<br>
<br />

In [192]:
def create_message(table_name, query):

    class message:
        def __init__(message, system, user, column_names, column_attr):
            message.system = system
            message.user = user
            message.column_names = column_names
            message.column_attr = column_attr

    
    system_template = """

    Given the following SQL table, your job is to write queries given a user’s request. Return just the SQL query as plan text, without additional text and don't use markdown format. \n

    CREATE TABLE {} ({}) \n
    """

    user_template = "Write a SQL query that returns - {}"
    
    tbl_describe = duckdb.sql("DESCRIBE SELECT * FROM " + table_name +  ";")
    col_attr = tbl_describe.df()[["column_name", "column_type"]]
    col_attr["column_joint"] = col_attr["column_name"] + " " +  col_attr["column_type"]
    col_names = str(list(col_attr["column_joint"].values)).replace('[', '').replace(']', '').replace('\'', '')

    system = system_template.format(table_name, col_names)
    user = user_template.format(query)

    m = message(system = system, user = user, column_names = col_attr["column_name"], column_attr = col_attr["column_type"])
    return m

**Note:** We will use the same prompt format as we used with for the OpenAI API, and modified it to the Gemini prompt format.

Let's test it:

In [193]:
query = "How many cases ended up with arrest?"
msg = create_message(table_name = "chicago_crime", query = query)

In [194]:
print(msg.system)



    Given the following SQL table, your job is to write queries given a user’s request. Return just the SQL query as plan text, without additional text and don't use markdown format. 


    CREATE TABLE chicago_crime (id BIGINT, case_number VARCHAR, datetime VARCHAR, block VARCHAR, iucr VARCHAR, primary_type VARCHAR, description VARCHAR, location_description VARCHAR, arrest BOOLEAN, domestic BOOLEAN, beat BIGINT, district BIGINT, ward DOUBLE, community_area DOUBLE, fbi_code VARCHAR, x_coordinate DOUBLE, y_coordinate DOUBLE, year BIGINT, updated_on TIMESTAMP_NS, latitude DOUBLE, longitude DOUBLE) 

    


In [195]:
prompt = [msg.system, msg.user]

In [196]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents= prompt,
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0,
        top_p = 1,
        top_k = 32
    )
)

In [197]:
code = response.candidates[0].content.parts[0].text
print(code)
code = extract_code_from_markdown(markdown_text=code)
print(code)


```sql
SELECT count(*) FROM chicago_crime WHERE arrest = TRUE
```
SELECT count(*) FROM chicago_crime WHERE arrest = TRUE


The next step is to set a function that send the prompt to the API (i.e., API handler). In this case, we will set the function to work with the Gemini API:

<figure>
 <img src="images/architecture 2.png" width="100%" align="center"/></a>
<figcaption> The agent architecture</figcaption>
</figure>

<br>
<br />

In [198]:
def lang2sql_gemini(api_key, table_name, query, model ="gemini-2.0-flash", temperature = 0,max_output_token = 256):
    class lang2sql:
        def __init__(output, message, response, sql):
            output.message = message
            output.response = response
            output.sql = sql      

    client = genai.Client(api_key = api_key)

    m = create_message(table_name = table_name, query = query)

    prompt = [m.system, m.user]

    response = client.models.generate_content(
    model=model,
    contents= prompt,
    config=types.GenerateContentConfig(
        max_output_tokens=max_output_token,
        temperature=temperature
        )
    )

    sql = response.candidates[0].content.parts[0].text
    if is_markdown_code_chunk(text = sql):
        sql = extract_code_from_markdown(markdown_text=sql)

    output = lang2sql(message = m, response = response, sql = sql)
    return output

Let's test it:

In [199]:
query = lang2sql_gemini(api_key = api_key, 
                        table_name = "chicago_crime", 
                        query = "How many cases ended up with arrest?")

In [200]:
print(query.sql)
duckdb.sql(query.sql).show()

SELECT count(*) FROM chicago_crime WHERE arrest = TRUE
┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        79921 │
└──────────────┘



Last but not least, we will set a wrapper function that generate the query and pull the data from the table:
<figure>
 <img src="images/architecture 3.png" width="100%" align="center"/></a>
<figcaption> The agent architecture</figcaption>
</figure>

<br>
<br />

In [201]:
def lang2answer_gemini(api_key, table_name, query, model ="gemini-2.0-flash", temperature = 0,max_output_token = 256, verbose = False):
    query = lang2sql_gemini(api_key = api_key,
                        table_name = table_name,
                        query = query, 
                        model = model,
                        temperature = temperature,
                        max_output_token = max_output_token
                    )
    if verbose:
        print(query.sql)
    output = duckdb.sql(query.sql)
    return output

Let's test it out!

In [202]:
lang2answer_gemini(api_key = api_key, 
            table_name = "chicago_crime", 
            query = "How many cases ended up with arrest?",
            verbose = True)

SELECT count(*) FROM chicago_crime WHERE arrest = TRUE


┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        79921 │
└──────────────┘

In [203]:
lang2answer_gemini(api_key = api_key, 
            table_name = "chicago_crime", 
            query = "Summarize the cases by primary type",
            verbose = True)

SELECT primary_type, COUNT(*) AS case_count FROM chicago_crime GROUP BY primary_type;


┌────────────────────────────┬────────────┐
│        primary_type        │ case_count │
│          varchar           │   int64    │
├────────────────────────────┼────────────┤
│ WEAPONS VIOLATION          │      18250 │
│ ROBBERY                    │      21977 │
│ MOTOR VEHICLE THEFT        │      55945 │
│ OBSCENITY                  │        116 │
│ PUBLIC PEACE VIOLATION     │       2150 │
│ CRIMINAL TRESPASS          │      11340 │
│ PROSTITUTION               │        567 │
│ OFFENSE INVOLVING CHILDREN │       3912 │
│ STALKING                   │       1203 │
│ OTHER NARCOTIC VIOLATION   │         10 │
│    ·                       │          · │
│    ·                       │          · │
│    ·                       │          · │
│ HOMICIDE                   │       1337 │
│ NON-CRIMINAL               │          8 │
│ LIQUOR LAW VIOLATION       │        453 │
│ KIDNAPPING                 │        269 │
│ GAMBLING                   │         39 │
│ CRIMINAL DAMAGE            │  

In [204]:
lang2answer_gemini(api_key = api_key, 
            table_name = "chicago_crime", 
            query = "Summarize the cases by primary type, sort by number of crimes in descending order",
            verbose = True)

SELECT primary_type, count(*) AS num_crimes FROM chicago_crime GROUP BY primary_type ORDER BY num_crimes DESC


┌───────────────────────────────────┬────────────┐
│           primary_type            │ num_crimes │
│              varchar              │   int64    │
├───────────────────────────────────┼────────────┤
│ THEFT                             │     134807 │
│ BATTERY                           │     103019 │
│ CRIMINAL DAMAGE                   │      66227 │
│ MOTOR VEHICLE THEFT               │      55945 │
│ ASSAULT                           │      52618 │
│ OTHER OFFENSE                     │      38416 │
│ DECEPTIVE PRACTICE                │      37709 │
│ ROBBERY                           │      21977 │
│ BURGLARY                          │      18314 │
│ WEAPONS VIOLATION                 │      18250 │
│         ·                         │         ·  │
│         ·                         │         ·  │
│         ·                         │         ·  │
│ CONCEALED CARRY LICENSE VIOLATION │        510 │
│ LIQUOR LAW VIOLATION              │        453 │
│ INTIMIDATION                 

Here is a case, the LLM provides an incorrect SQL syntax:


In [205]:
lang2answer_gemini(api_key = api_key, 
                        table_name = "chicago_crime", 
                        query = "How many cases is the type of robbery?",
                        verbose = True)

SELECT count(*) FROM chicago_crime WHERE primary_type = "ROBBERY"


BinderException: Binder Error: Referenced column "ROBBERY" not found in FROM clause!
Candidate bindings: "id"

The error is due to the use of `"` instead of `''` for the `ROBBERY` name in the query:

```sql
SELECT count(*) FROM chicago_crime WHERE primary_type = "ROBBERY"
```

This is the correct query:

In [207]:
sql = "SELECT count(*) FROM chicago_crime WHERE primary_type = 'ROBBERY'"
duckdb.sql(sql).show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        21977 │
└──────────────┘



In [208]:
lang2answer_gemini(api_key = api_key, 
            table_name = "chicago_crime", 
            query = "Show me cases that are type of robbery", 
            verbose=True)

SELECT * FROM chicago_crime WHERE primary_type = 'ROBBERY'


┌──────────┬─────────────┬─────────────────────┬────────────────────────┬─────────┬──────────────┬────────────────────────────────────┬──────────────────────┬─────────┬──────────┬───────┬──────────┬────────┬────────────────┬──────────┬──────────────┬──────────────┬───────┬─────────────────────┬──────────────┬───────────────┐
│    id    │ case_number │      datetime       │         block          │  iucr   │ primary_type │            description             │ location_description │ arrest  │ domestic │ beat  │ district │  ward  │ community_area │ fbi_code │ x_coordinate │ y_coordinate │ year  │     updated_on      │   latitude   │   longitude   │
│  int64   │   varchar   │       varchar       │        varchar         │ varchar │   varchar    │              varchar               │       varchar        │ boolean │ boolean  │ int64 │  int64   │ double │     double     │ varchar  │    double    │    double    │ int64 │    timestamp_ns     │    double    │    double     │
├──────────┼───────